# 1. Load synthetic notes

In [ ]:
import pandas as pd
import json

df = pd.read_csv("data/synthetic_notes.csv")

# 2. Extract data from our notes
Code for [I Ran OpenAI’s New Open Model on My Laptop to Extract Medical Data — Here’s What Happened](https://medium.com/towards-artificial-intelligence/i-ran-openais-new-open-model-on-my-laptop-to-extract-medical-data-here-s-what-happened-aeb6acddfede)

In [23]:
from discharge_agent.extractions.extraction import extract_clinical_information

results = extract_clinical_information(df['note_text'][2])

In [24]:
result_json = json.loads(results)

In [ ]:
with open("data/processed_notes/diabetic_ketoacidosis.json", "w") as f:
    json.dump(result_json, f)

# 3. Run discharge checks
Demo code for [I Built a Local Clinical AI Agent from Scratch - Here's How]()

In [1]:
import json

with open("data/processed_notes/upper_gi_bleed.json", "r") as f:
    result_json = json.load(f)

In [2]:
result_json

{'discharge_date': '2024-08-25',
 'chief_complaint': 'Black tarry stools',
 'primary_discharge_diagnosis': 'Upper GI bleeding 2/2 duodenal ulcer, acute blood loss anemia',
 'procedures_performed': ['EGD with thermal therapy to bleeding duodenal ulcer'],
 'discharge_condition': {'mental_status': 'Clear and coherent',
  'consciousness_level': 'Alert and interactive',
  'activity_status': 'Ambulatory - independent'},
 'discharge_disposition': 'Home',
 'medication_changes': {'new_medications': [{'name': 'Omeprazole',
    'dose': '40mg',
    'frequency': 'PO BID'},
   {'name': 'Sucralfate', 'dose': '1g', 'frequency': 'PO QID'},
   {'name': 'Iron sulfate', 'dose': '325mg', 'frequency': 'PO daily'},
   {'name': 'Acetaminophen', 'dose': '650mg', 'frequency': 'PO Q6H PRN pain'}],
  'dose_changes': []},
 'follow_up_appointments': [{'provider': 'Andrew Peterson',
   'specialty': 'GI',
   'date': '2024-09-05',
   'time': '2:15 PM'},
  {'provider': '', 'specialty': '', 'date': '2024-10-18', 'time':

In [5]:
from discharge_agent.pipelines.discharge_checker import check_discharge_safety
from discharge_agent.llm.prompts import get_messages

messages = get_messages(result_json)
final = check_discharge_safety(messages)

{'function': {'name': 'flag_labs', 'arguments': {'labs': [{'date': '2024-08-24', 'name': 'Hgb', 'value': '10.2'}, {'date': '2024-08-24', 'name': 'Hct', 'value': '31%'}, {'date': '2024-08-24', 'name': 'Plt', 'value': '245'}, {'date': '2024-08-24', 'name': 'BUN', 'value': '25'}, {'date': '2024-08-24', 'name': 'Cr', 'value': '1.1'}]}}}
{'function': {'name': 'umls_normalize', 'arguments': {'terms': ['Upper gastrointestinal bleeding', 'duodenal ulcer', 'acute blood loss anemia']}}}
{'function': {'name': 'followup_gap', 'arguments': {'appts': [{'date': '2024-09-05', 'provider': 'Andrew Peterson', 'specialty': 'GI', 'time': '2:15 PM'}, {'date': '2024-10-18', 'provider': '', 'specialty': '', 'time': '11:00 AM'}], 'discharge_date': '2024-08-25'}}}
{ "ready": false, "reasons": ["Abnormal labs: Hgb low, Hct low, BUN high", "Follow-up appointment >7 days"], "summary": { "labs": { "abnormal": [ { "name_input": "Hgb", "name_norm": "hemoglobin", "value": "10.2", "value_num": 10.2, "ref_range": "12.0-

In [6]:
json.loads(final)

{'ready': False,
 'reasons': ['Abnormal labs: Hgb low, Hct low, BUN high',
  'Follow-up appointment >7 days'],
 'summary': {'labs': {'abnormal': [{'name_input': 'Hgb',
     'name_norm': 'hemoglobin',
     'value': '10.2',
     'value_num': 10.2,
     'ref_range': '12.0-16.0',
     'status': 'low'},
    {'name_input': 'Hct',
     'name_norm': 'hematocrit',
     'value': '31%',
     'value_num': 31.0,
     'ref_range': '36.0-50.0',
     'status': 'low'},
    {'name_input': 'BUN',
     'name_norm': 'bun',
     'value': '25',
     'value_num': 25.0,
     'ref_range': '7.0-24.0',
     'status': 'high'}],
   'ok': False},
  'followup': {'days_to_earliest_followup': 11, 'meets_goal': False},
  'meds': {'new_medications': [{'name': 'Omeprazole',
     'dose': '40mg',
     'frequency': 'PO BID'},
    {'name': 'Sucralfate', 'dose': '1g', 'frequency': 'PO QID'},
    {'name': 'Iron sulfate', 'dose': '325mg', 'frequency': 'PO daily'},
    {'name': 'Acetaminophen',
     'dose': '650mg',
     'frequen